In [1]:
import random
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# POIS

In [ ]:
MATRIX_COL = 1000
MATRIX_ROW = 10000

In [ ]:
def prost_potok(lmbd):
  time_event = random.expovariate(lmbd)
  return(time_event)

In [ ]:
time_list = np.zeros([MATRIX_ROW, MATRIX_COL])
lmbd_list = []

for i in range(MATRIX_ROW):
  lmbd = random.uniform(0, 10000)
  lmbd_list.append(lmbd)
  counter = 0
  T = 0
  time = 0
  for j in range(MATRIX_COL):
    T = prost_potok(lmbd)
    time += T
    time_list[i][j] = time
    counter += 1
    
df = pd.DataFrame(time_list)
df['lmbd_value'] = lmbd_list

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,lmbd_value
0,0.000026,0.000040,0.000193,0.000196,0.000280,0.000996,0.001051,0.001162,0.001181,0.001194,8130.165887
1,0.000226,0.000312,0.000791,0.001390,0.001947,0.002038,0.002345,0.002484,0.002713,0.003071,3327.570523
2,0.000219,0.000606,0.000851,0.000856,0.001316,0.001384,0.001799,0.001824,0.001921,0.002022,5912.330378
3,0.000011,0.000133,0.000307,0.000402,0.000434,0.000479,0.000622,0.001000,0.001002,0.001019,8490.209204
4,0.000362,0.000586,0.000934,0.001090,0.001562,0.001699,0.001722,0.001883,0.001954,0.001999,3134.589386


In [ ]:
df.to_csv('/content/drive/MyDrive/maga/dataset/dataset_pois.csv', sep=',')

# MMPP

In [2]:
MATRIX_COL = 15
MATRIX_ROW = 2

In [3]:
def sum_diag_matrix(matrix, matrix_size):
  for i in range(matrix_size):
    for j in range(matrix_size):
      if i==j:
        matrix[i, j] = -(sum(matrix[i]) - matrix[i, j])
  return matrix

def calculation_pi(q_matrix_for_solve, matrix_size):
    v = np.zeros(matrix_size)
    v[-1] = 1
    pi = np.linalg.solve(q_matrix_for_solve, v)
    return pi

def choose_time_work(pi, T, K,k):
    p = 0
    rnd = random.random()

    for i in range (len(pi)):
      if rnd >= (p +pi[i]):
        p += pi[i]
      else: 
        k = i
    rnd_list = []
    for v in range(K):
        if v != k:
            rnd = random.expovariate(q_matrix[k][v])
            rnd_list.append(rnd)
        if v == k:
            rnd = 10e100
            rnd_list.append(rnd)
    temp = min(rnd_list)
    k = rnd_list.index(min(rnd_list))
    T += temp
    return k, T

def cycle(T, k, s, t0, time_list):
    while s < S:
        thau = random.expovariate(lmbd_matrix[k])
        if t0 + thau < T:
            t0 += thau
            s += 1
            time_list.append(t0)
        else:
            return s, T, time_list
    return s, T, time_list

In [4]:
k_list = []
q_matrix_list = []
lmbd_matrix_list = []

t_time = []
for i in range(MATRIX_ROW):
  matrix_size = random.randint(2, 10)
  k_list.append(matrix_size)
  
  q_matrix = np.random.uniform(0, 10000, size=(matrix_size, matrix_size))
  q_matrix = sum_diag_matrix(q_matrix, matrix_size)
  q_matrix_list.append(np.asarray(q_matrix).reshape(-1))

  q_matrix_for_solve = q_matrix.copy()
  q_matrix_for_solve[matrix_size-1] = np.ones(matrix_size)

  pi = calculation_pi(q_matrix_for_solve, matrix_size)

  lmbd_matrix = np.random.uniform(0, 10000, matrix_size)
  lmbd_matrix_list.append(np.asarray(lmbd_matrix).reshape(-1))

  log_item = 0
  p = 0
  K = matrix_size                                                  
  v = 0
  S = MATRIX_COL
  T = 0
  s = 0
  t0 = 0
  time_list=[]

  k = 0

  while s < S:
    k, T = choose_time_work(pi, T, K, k)
    s, t0, time_list = cycle(T, k, s, t0, time_list)

  t_time.append(time_list)

In [ ]:
q_matrix_df = pd.DataFrame(q_matrix_list).fillna(0)
q_matrix_df.columns = q_matrix_df.columns.astype('str')
q_matrix_df.columns = 'q_' + q_matrix_df.columns
q_matrix_df = q_matrix_df.reset_index()

lmbd_matrix_df = pd.DataFrame(lmbd_matrix_list).fillna(0)
lmbd_matrix_df.columns = lmbd_matrix_df.columns.astype('str')
lmbd_matrix_df.columns = 'lmbd_' + lmbd_matrix_df.columns
lmbd_matrix_df = lmbd_matrix_df.reset_index()

lmbd_matrix_df['k_list'] = k_list

df = pd.DataFrame(t_time).reset_index()

df = df.merge(q_matrix_df, how='left', on='index')
df = df.merge(lmbd_matrix_df, how='left', on='index')
df = df.drop('index', axis = 1)

In [ ]:
df.to_csv('/content/drive/MyDrive/maga/dataset/dataset_mmpp.csv', sep=',')

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,lmbd_1,lmbd_2,lmbd_3,lmbd_4,lmbd_5,lmbd_6,lmbd_7,lmbd_8,lmbd_9,k_list
0,0.000097,0.000208,0.000246,0.000290,0.000356,0.000671,0.000674,0.000683,0.000952,0.001123,...,1423.179006,6266.955483,7614.954926,2263.551178,9594.869412,4.921381,3731.334325,0.000000,0.00000,8
1,0.000151,0.000151,0.000273,0.000589,0.001029,0.001208,0.001673,0.001767,0.001920,0.002077,...,8132.744553,2342.228915,4227.869617,2404.119922,1077.043402,0.000000,0.000000,0.000000,0.00000,6
2,0.000324,0.000717,0.000910,0.001786,0.002712,0.003155,0.003460,0.003552,0.003928,0.004128,...,3360.598970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2
3,0.000111,0.000889,0.001159,0.001371,0.001484,0.001486,0.001499,0.001784,0.001796,0.001827,...,504.343071,8029.679318,3909.981739,5961.889839,3394.340421,4735.988677,3792.402876,1517.297099,7343.19947,10
4,0.000043,0.000242,0.000274,0.000644,0.001195,0.003421,0.003451,0.003667,0.003758,0.004359,...,4211.580631,684.116738,1799.257416,6128.556611,3029.529671,4508.758749,2545.631452,0.000000,0.00000,8


# RECCUR

In [ ]:
MATRIX_COL = 3000
MATRIX_ROW = 10000

In [ ]:
def reccur_potok(alpha, beta):
  time_event = random.gammavariate(alpha, beta)
  return(time_event)

In [ ]:
time_list = np.zeros([MATRIX_ROW, MATRIX_COL])

In [ ]:
alpha_list = [random.uniform(0, 100) for _ in range(MATRIX_ROW)]
beta_list = [random.uniform(0, 100) for _ in range(MATRIX_ROW)]

In [ ]:
#time_list = np.zeros([MATRIX_ROW, MATRIX_COL])
#alpha_list = [random.uniform(0, 10000) for _ in range(MATRIX_ROW)]
#sq = 1/(max(alpha_list) * 5000)
#beta_list = [random.uniform(1/(alpha_list[i] * 5000)-sq, 1/(alpha_list[i] * 5000)+sq) for i in range(MATRIX_ROW)]

for i in range(MATRIX_ROW):
  counter = 0
  T = 0
  time = 0
  for j in range(MATRIX_COL):
    T = reccur_potok(alpha_list[i], beta_list[i])
    time += T
    time_list[i][j] = time
    counter += 1
    
df = pd.DataFrame(time_list)
df['alpha_value'] = alpha_list
df['beta_value'] = beta_list

In [ ]:
df['lmbd'] = 1/(df['alpha_value'] * df['beta_value'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(df['lmbd'])

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,2993,2994,2995,2996,2997,2998,2999,alpha_value,beta_value,lmbd
0,2630.310859,5853.436059,8540.277915,11260.504628,13774.235095,16650.107737,19510.573387,21908.946569,25074.571613,28048.955108,...,8.341865e+06,8.344665e+06,8.346871e+06,8.349607e+06,8.352697e+06,8.355120e+06,8.358050e+06,91.139033,30.499855,0.000360
1,1644.375584,3403.045546,4895.019485,6134.983863,8231.790363,9802.673147,11133.086113,13335.481816,15434.711492,17608.711299,...,4.767325e+06,4.769053e+06,4.770522e+06,4.772063e+06,4.773385e+06,4.775080e+06,4.776913e+06,29.804622,53.638389,0.000626
2,4754.732433,10043.542091,14958.889990,19639.452993,25716.978683,30429.508608,35239.872894,39828.221087,45155.768056,50629.221154,...,1.439432e+07,1.439840e+07,1.440368e+07,1.440844e+07,1.441308e+07,1.441803e+07,1.442225e+07,90.920112,52.820400,0.000208
3,5457.227695,10579.134694,15924.685545,20774.435431,26714.676459,32826.864797,38472.651677,43158.831724,48903.018374,53144.841467,...,1.542433e+07,1.542910e+07,1.543356e+07,1.543875e+07,1.544298e+07,1.544865e+07,1.545272e+07,60.142273,85.603451,0.000194
4,281.717209,590.167386,730.716721,775.828064,1182.021627,1310.777666,1624.033725,1718.228740,1902.623807,1955.833694,...,6.213266e+05,6.214957e+05,6.218667e+05,6.220795e+05,6.221878e+05,6.223871e+05,6.229958e+05,2.490102,82.369009,0.004875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,462.617874,1036.710853,1512.507814,1993.906036,2607.088186,2928.826354,3414.511573,3946.738034,4548.990987,4998.522450,...,1.426794e+06,1.427355e+06,1.427855e+06,1.428342e+06,1.428695e+06,1.429116e+06,1.429620e+06,17.413140,27.491903,0.002089
9996,2629.541494,5435.453059,7572.196791,10298.040022,13013.664138,16186.488025,18595.762814,21304.565859,23706.349621,26467.416521,...,8.579458e+06,8.582291e+06,8.585643e+06,8.588949e+06,8.591648e+06,8.593918e+06,8.597529e+06,52.337858,54.695269,0.000349
9997,2117.218877,4599.338259,7252.307695,8890.481642,10990.641921,13232.183474,14865.961995,16401.853773,18482.140056,20363.073218,...,6.247135e+06,6.249522e+06,6.251701e+06,6.253491e+06,6.255799e+06,6.257611e+06,6.259366e+06,54.421412,38.337270,0.000479
9998,6771.886297,12588.779980,17850.922000,22342.766326,27545.925435,33206.172272,37891.740811,43106.420717,47713.380998,53158.059042,...,1.518606e+07,1.519147e+07,1.519670e+07,1.520197e+07,1.520697e+07,1.521255e+07,1.521770e+07,92.207849,54.855396,0.000198


In [ ]:
df.to_csv('/content/drive/MyDrive/maga/dataset/dataset_recurr4.csv', sep=',')